In [1]:
#Intro

In [2]:
#import libraries to be used 

#general
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

#visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt


import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#from textblob import TextBlob

#machine learning model requirements
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 
from sklearn import metrics
from sklearn.metrics import (
    f1_score, roc_auc_score, accuracy_score,
    precision_recall_curve, roc_curve, average_precision_score
)
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack


#models
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


In [3]:
import os
import pandas as pd

# List all files in the BABE dataset
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.xlsx'):
            excel_path = os.path.join(dirname, filename)
            print("Excel file found:", excel_path)

            # List all sheet names in that file
            xls = pd.ExcelFile(excel_path)
            print("Sheet names:", xls.sheet_names)


Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/final_labels_SG1.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/final_labels_SG2.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/dt_final_SG1.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/final_labels_MBIC.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/raw_labels_SG1.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/dt_final_SG2.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/raw_labels_MBIC.xlsx
Sheet names: ['annotations']
Excel file found: /kaggle/input/babe-media-bias-annotations-by-experts/data/raw_labels_SG2.xlsx
Sheet names: ['Sheet1']
Excel file found: /kaggle/input

In [4]:
# Load just one worksheet by name
df = pd.read_excel('/kaggle/input/babe-media-bias-annotations-by-experts/data/dt_final_SG2.xlsx', sheet_name='Sheet1')


In [5]:
df.columns

Index(['Unnamed: 0', 'sentence', 'outlet', 'topic', 'type', 'article',
       'biased_words2', 'text', 'text_low', 'pos',
       ...
       'ne_NORP_context', 'ne_ORDINAL_context', 'ne_ORG_context',
       'ne_PERCENT_context', 'ne_PERSON_context', 'ne_PRODUCT_context',
       'ne_QUANTITY_context', 'ne_TIME_context', 'ne_WORK_OF_ART_context',
       'ne_LANGUAGE_context'],
      dtype='object', length=301)

In [6]:
def evaluate_file(file_path, duplicate_column=None):
    """
    Evaluates a given CSV or Excel file by:
    - Printing general file information
    - Checking for duplicate values in a specified column
    - Searching for zero values
    - Searching for empty (NaN) cells
    """

    # Print file information
    print("\n=== File Info ===")
    print(file_path.info())

    # Check for duplicates in the specified column
    if duplicate_column:
        duplicate_count = file_path.duplicated(subset=[duplicate_column]).sum()
        print(f"\n=== Duplicates in '{duplicate_column}' ===")
        print(f"Total duplicate values: {duplicate_count}")
    
    # Check for zero values
    zero_values = (file_path == 0).sum().sum()
    print(f"\n=== Zero Values ===")
    print(f"Total zero values: {zero_values}")

    # Check for empty (NaN) cells
    missing_values = file_path.isnull().sum().sum()
    print(f"\n=== Empty (NaN) Cells ===")
    print(f"Total empty cells: {missing_values}")
    
    #print a sample 
    display(file_path.sample(n=5))

In [7]:
#Run pipeline for each of the files to evaluate issues 
evaluate_file(df, duplicate_column="text")  


=== File Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65822 entries, 0 to 65821
Columns: 301 entries, Unnamed: 0 to ne_LANGUAGE_context
dtypes: bool(2), float64(4), int64(281), object(14)
memory usage: 150.3+ MB
None

=== Duplicates in 'text' ===
Total duplicate values: 52531

=== Zero Values ===
Total zero values: 17437766

=== Empty (NaN) Cells ===
Total empty cells: 140258


,Unnamed: 0,sentence,outlet,topic,type,article,biased_words2,text,text_low,pos,...,ne_NORP_context,ne_ORDINAL_context,ne_ORG_context,ne_PERCENT_context,ne_PERSON_context,ne_PRODUCT_context,ne_QUANTITY_context,ne_TIME_context,ne_WORK_OF_ART_context,ne_LANGUAGE_context
5011,8322,An analysis by Fox News shows at least eight p...,Fox News,middle-class,right,An analysis by Fox News shows at least eight p...,"['drastic', 'overhaul']",system,system,NOUN,...,0,0,0,0,0,0,0,0,0,0
53726,89616,The United States should learn from China and ...,Reuters,gun control,center,The United States should learn from China and ...,[],China,china,PROPN,...,0,0,0,0,0,0,0,0,0,0
29151,48769,Leonhardt’s column in the New York Times provi...,Breitbart,middle-class,right,Leonhardt’s column in the New York Times provi...,"['enamored', 'chaotic', 'fractured']",column,column,NOUN,...,0,0,1,0,0,0,0,0,0,0
64523,108111,White supremacist violent extremists can gener...,MSNBC,white-nationalism,left,White supremacist violent extremists can gener...,"['violent', 'virulent', 'supremacist', 'hatred']",anti,anti,NOUN,...,1,0,0,0,0,0,0,0,0,0
150,238,[The police] now prefer to think of themselves...,Breitbart,marriage-equality,NaN,[The police] now prefer to think of themselves...,['wounds'],prance,prance,VERB,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Show columns with at least one NaN value
nan_columns = df.columns[df.isna().any()].tolist()

print("Columns with NaN values:", nan_columns)


Columns with NaN values: ['type', 'tfidf_art', 'ne_label', 'MRCP_concretness_ratings', 'MRCP_Imagability_ratings']


In [9]:

# identify which columns have one or more zeroes
zero_columns = df.columns[(df == 0).any()].tolist()

print("Columns containing zeroes:", zero_columns)

Columns containing zeroes: ['Unnamed: 0', 'is_stop', 'order', 'tfidf_art', 'label3', 'label4', 'label5', 'is_ne', 'negative_conc', 'positive_conc', 'weak_subj', 'strong_subj', 'hyperbolic_terms', 'attitude_markers', 'kill_verbs', 'bias_lexicon', 'assertive_verbs', 'factive_verbs', 'report_verbs', 'implicative_verbs', 'hedges', 'boosters', 'affect ', 'posemo ', 'negemo ', 'anx ', 'anger ', 'sad ', 'social ', 'family ', 'friend ', 'female ', 'male ', 'cogproc ', 'insight ', 'cause ', 'discrep ', 'tentat ', 'certain ', 'differ ', 'percept ', 'see ', 'hear ', 'feel ', 'bio ', 'body ', 'health ', 'sexual ', 'ingest ', 'drives ', 'affiliation ', 'achieve ', 'power ', 'reward ', 'risk ', 'focuspast ', 'focuspresent ', 'focusfuture ', 'relativ ', 'motion ', 'space ', 'time ', 'work ', 'leisure ', 'home ', 'money ', 'relig ', 'death ', 'informal ', 'swear ', 'netspeak ', 'assent ', 'nonflu ', 'filler ', 'pos_ADJ', 'pos_ADP', 'pos_ADV', 'pos_AUX', 'pos_DET', 'pos_INTJ', 'pos_NOUN', 'pos_PRON', '

In [10]:
# List the columns you want to process to fill since empty
columns_to_fill = ['tfidf_art', 'MRCP_concretness_ratings', 'MRCP_Imagability_ratings',]

# Fill NaN values in each column with that column's mean
for col in columns_to_fill:
    mean_val = df[col].mean()
    df[col] = df[col].fillna(mean_val)


In [11]:
# columns to process
columns_to_clean = ['ne_label', 'type']

# Replace [''], [], or NaN with ['none'] in each column
for col in columns_to_clean:
    df[col] = df[col].apply(lambda x: ['none'] if (x == [''] or x == [] or pd.isna(x)) else x)

In [12]:
print(df['biased_words2'].value_counts())

biased_words2
[]                              33655
['claiming']                      439
['claimed']                       316
['claims']                        222
['illegal', 'aliens']             209
                                ...  
['Firebombs']                       5
['bungling']                        5
['destructive']                     4
['there', "isn't", 'anyone']        3
['poison']                          3
Name: count, Length: 1623, dtype: int64


In [13]:
#check to see that all NAN cells have been filled. 
evaluate_file(df, duplicate_column="text")  


=== File Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65822 entries, 0 to 65821
Columns: 301 entries, Unnamed: 0 to ne_LANGUAGE_context
dtypes: bool(2), float64(4), int64(281), object(14)
memory usage: 150.3+ MB
None

=== Duplicates in 'text' ===
Total duplicate values: 52531

=== Zero Values ===
Total zero values: 17437766

=== Empty (NaN) Cells ===
Total empty cells: 0


,Unnamed: 0,sentence,outlet,topic,type,article,biased_words2,text,text_low,pos,...,ne_NORP_context,ne_ORDINAL_context,ne_ORG_context,ne_PERCENT_context,ne_PERSON_context,ne_PRODUCT_context,ne_QUANTITY_context,ne_TIME_context,ne_WORK_OF_ART_context,ne_LANGUAGE_context
32706,54621,No sooner do states start getting together to ...,Alternet,gun control,left,No sooner do states start getting together to ...,['assert'],start,start,VERB,...,0,0,0,0,0,0,0,0,0,0
4645,7737,American Outdoor Brands Corp AOBC.O said on Th...,Reuters,gun control,center,American Outdoor Brands Corp AOBC.O said on Th...,[],said,said,VERB,...,0,0,1,0,0,0,0,0,0,0
2217,3644,A Supreme Court filing lays bare the deep chas...,Alternet,taxes,left,A Supreme Court filing lays bare the deep chas...,['wannabe'],lays,lays,VERB,...,0,0,1,0,0,0,0,0,0,0
55548,92804,This dark new event revealed that America’s wh...,Alternet,white-nationalism,left,This dark new event revealed that America’s wh...,['dark'],America,america,PROPN,...,0,0,0,0,0,0,0,0,0,0
13535,22530,Conservative Republican Senator Ted Cruz favor...,Reuters,universal health care,center,Conservative Republican Senator Ted Cruz favor...,[],standards,standards,NOUN,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.head(35)

,Unnamed: 0,sentence,outlet,topic,type,article,biased_words2,text,text_low,pos,...,ne_NORP_context,ne_ORDINAL_context,ne_ORG_context,ne_PERCENT_context,ne_PERSON_context,ne_PRODUCT_context,ne_QUANTITY_context,ne_TIME_context,ne_WORK_OF_ART_context,ne_LANGUAGE_context
0,0,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],Orange,orange,PROPN,...,0,0,0,0,0,0,0,0,1,0
1,3,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],New,new,PROPN,...,0,0,0,0,0,0,0,0,1,0
2,4,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],Black,black,PROPN,...,0,0,0,0,1,0,0,0,1,0
3,5,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],star,star,NOUN,...,0,0,0,0,1,0,0,0,1,0
4,6,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],Yael,yael,PROPN,...,0,0,0,0,1,0,0,0,1,0
5,7,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],Stone,stone,PROPN,...,0,0,0,0,1,0,0,0,0,0
6,9,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],renouncing,renouncing,VERB,...,0,0,0,0,1,0,0,0,0,0
7,11,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],U.S.,u.s.,PROPN,...,0,0,0,0,0,0,0,0,0,0
8,12,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],green,green,ADJ,...,0,0,0,0,0,0,0,0,0,0
9,13,"""Orange Is the New Black"" star Yael Stone is r...",Fox News,environment,right,"""Orange Is the New Black"" star Yael Stone is r...",[],card,card,NOUN,...,0,0,0,0,1,0,0,0,0,0


In [15]:
#find all columns with words
def contains_words(series):
    return series.astype(str).apply(lambda x: any(char.isalpha() for char in x)).any()

word_columns = [col for col in df.columns if contains_words(df[col])]

print("Columns that contain words:", word_columns)

Columns that contain words: ['sentence', 'outlet', 'topic', 'type', 'article', 'biased_words2', 'text', 'text_low', 'pos', 'lemma', 'lemma_low', 'tag', 'dep', 'is_stop', 'is_ne', 'ne_label']


In [16]:
#check datatypes
print(df.dtypes)

Unnamed: 0                 int64
sentence                  object
outlet                    object
topic                     object
type                      object
                           ...  
ne_PRODUCT_context         int64
ne_QUANTITY_context        int64
ne_TIME_context            int64
ne_WORK_OF_ART_context     int64
ne_LANGUAGE_context        int64
Length: 301, dtype: object


In [17]:
#above is too broad so be more specific
def contains_list_or_string(col):
    return col.apply(lambda x: isinstance(x, (list, str))).any()

list_or_string_columns = [col for col in df.columns if contains_list_or_string(df[col])]

print("Columns where any value is a list or string:", list_or_string_columns)


Columns where any value is a list or string: ['sentence', 'outlet', 'topic', 'type', 'article', 'biased_words2', 'text', 'text_low', 'pos', 'lemma', 'lemma_low', 'tag', 'dep', 'ne_label']


In [18]:
# List of columns to clean and encode
columns_to_encode = list_or_string_columns

label_encoders = {}

# Step 1: Normalize the data: convert lists to strings
for col in columns_to_encode:
    df[col] = df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Apply LabelEncoder
for col in columns_to_encode:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str).fillna('unknown'))
    label_encoders[col] = le


In [19]:
# Vectorize multiple text columns
tfidf_vectorizers = {}
tfidf_matrices = []

for col in word_columns:
    tfidf = TfidfVectorizer(stop_words='english', max_features=100)
    matrix = tfidf.fit_transform(df[col].fillna('').astype(str))  # handle NaNs + force to string
    tfidf_vectorizers[col] = tfidf
    tfidf_matrices.append(matrix)

# Combine all text vectors into one feature matrix
from scipy.sparse import hstack
X_text_combined = hstack(tfidf_matrices)

#define target 
y = df['type'] 
#drop target so not leak into features
df = df.drop('type', axis=1)

# Pull out all encoded numeric columns
encoded_features_array = df[[col + '_encoded' for col in columns_to_encode]].values

# Combine sparse TF-IDF matrix with dense numeric matrix
X_combined = hstack([X_text_combined, encoded_features_array])

In [20]:
#train/test
X_train, X_temp, y_train, y_temp = train_test_split(X_combined, y, test_size=0.4, random_state=54321)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=54321)

# Assign 
features_train = X_train
target_train = y_train

features_valid = X_valid
target_valid = y_valid

features_test = X_test
target_test = y_test

In [ ]:
# Define models and their respective parameter grids
models_and_params = {
    'RandomForest': {
        'model': RandomForestClassifier(),
        'param_grid': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'random_state': [42]
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'param_grid': {
            'max_depth': [3, 5, 10],
            'random_state': [42]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingClassifier(),
        'param_grid': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 10],
            'random_state': [42]
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(),
        'param_grid': {}
    },
    'XGBClassifier': {
        'model': xgb.XGBClassifier(),
        'param_grid': {} 
    },
    'LGBMClassifier': {
        'model': lgb.LGBMClassifier(),
        'param_grid': {}
     },
    'CatBoost': {
        'model': CatBoostClassifier(),
        'param_grid': {}
    },
}

def train_models(models_and_params, features_train, target_train):
    trained_models = {}

    for model_name, config in models_and_params.items():
        print(f"\nRunning GridSearchCV for {model_name}...")

        # Hyperparameter tuning
        grid_search = GridSearchCV(
            estimator=config['model'],
            param_grid=config['param_grid'],
            cv=5,
            scoring='f1_macro',
            n_jobs=-1
        )
        grid_search.fit(features_train, target_train)

        # Store the best model
        best_model = grid_search.best_estimator_
        trained_models[model_name] = best_model
        print(f"Best {model_name} Parameters: {grid_search.best_params_}")

    return trained_models

def evaluate_models(trained_models, features_train, target_train, features_test, target_test):
    for model_name, model in trained_models.items():
        print(f"\nEvaluating {model_name}...")
        evaluate_classification_model(model_name, model, features_train, target_train, features_test, target_test)

def evaluate_classification_model(model_name, model, features_train, target_train, features_test, target_test):
    eval_stats = {}
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))

    for dataset_type, features, target in [('train', features_train, target_train), ('test', features_test, target_test)]:
        eval_stats[dataset_type] = {}

        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]  # Use the correct dataset

        # Compute metrics
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [f1_score(target, pred_proba >= threshold) for threshold in f1_thresholds]
        fpr, tpr, _ = roc_curve(target, pred_proba)
        precision, recall, _ = precision_recall_curve(target, pred_proba)

        # Aggregate results
        eval_stats[dataset_type]['Accuracy'] = accuracy_score(target, pred_target)
        eval_stats[dataset_type]['F1'] = f1_score(target, pred_target)
        eval_stats[dataset_type]['ROC AUC'] = roc_auc_score(target, pred_proba)
        eval_stats[dataset_type]['APS'] = average_precision_score(target, pred_proba)

        color = 'blue' if dataset_type == 'train' else 'green'

        # F1 Score Plot
        ax = axs[0]
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{dataset_type}, max={max(f1_scores):.2f}')
        ax.set_xlabel('Threshold')
        ax.set_ylabel('F1 Score')
        ax.legend()
        ax.set_title(f'F1 Score ({model_name})')

        # ROC Curve
        ax = axs[1]
        ax.plot(fpr, tpr, color=color, label=f'{dataset_type}, AUC={roc_auc_score(target, pred_proba):.2f}')
        ax.plot([0, 1], [0, 1], linestyle='--', color='gray')
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.legend()
        ax.set_title(f'ROC Curve ({model_name})')
                # Precision-Recall Curve
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{dataset_type}, AP={average_precision_score(target, pred_proba):.2f}')
        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.legend()
        ax.set_title(f'Precision-Recall Curve ({model_name})')

    df_eval_stats = pd.DataFrame(eval_stats).round(2)
    df_eval_stats = df_eval_stats.reindex(index=['Accuracy', 'F1', 'APS', 'ROC AUC'])

    print(df_eval_stats)
    plt.show()

# Run training and evaluation separately
trained_models = train_models(models_and_params, features_train, target_train)
evaluate_models(trained_models, features_train, target_train, features_test, target_test)


In [ ]:
def find_best_model_by_auc(trained_models, features_test, target_test):
    """Find the model with the highest ROC-AUC score on the test set."""
    best_model = None
    best_model_name = None
    best_auc = 0

    for model_name, model in trained_models.items():
        pred_proba = model.predict_proba(features_test)[:, 1]
        auc = roc_auc_score(target_test, pred_proba)

        print(f"{model_name} Test ROC-AUC: {auc:.4f}")

        if auc > best_auc:
            best_auc = auc
            best_model = model
            best_model_name = model_name

    print(f"\nBest Model: {best_model_name} with ROC-AUC: {best_auc:.4f}")
    return best_model, best_model_name

# Find the best model based on ROC-AUC
best_model, best_model_name = find_best_model_by_auc(trained_models, features_test, target_test)

# Compute ROC-AUC on the validation set using the best model
best_val_pred_proba = best_model.predict_proba(features_valid)[:, 1]
auc_roc_val = roc_auc_score(target_valid, best_val_pred_proba)

print(f"AUC-ROC Score for Best Model ({best_model_name}) on Validation Set: {auc_roc_val:.4f}")
